# Akur8 International Week: From Single LLM Calls to Agents

**Welcome!** In the next 2 hours, you'll go from zero to building your own AI agent.

No coding experience required. Just run each cell in order and follow along.

### What You'll Learn

1. **Single LLM calls** - Send a prompt, get a response
2. **Structured output** - Get data, not just text
3. **Workflows** - Chain multiple calls together
4. **Tool calling** - Give the LLM actions it can take
5. **Agents** - LLMs that decide their own steps

By the end, you'll build something fun with all of this. But first, let's learn the fundamentals.


# Part 1: The Capability Ladder


### Setup

Run this cell first. It installs what we need and sets up the connection to the AI model.

▶️ **Run the cell below**


In [ ]:
%pip install anthropic fastapi uvicorn --quiet

!mkdir -p vending
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/helpers.py -O vending/helpers.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/__init__.py -O vending/__init__.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/simulation.py -O vending/simulation.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/server.py -O vending/server.py

import json
from vending import helpers
from vending.server import launch_simulation

# Workshop API key (provided by facilitator)
API_KEY = ""

helpers.init(API_KEY)

print("✅ Setup complete! You're ready to go.")

### Step 1: The Single LLM Call

The simplest thing you can do with an LLM is send a message in, get a response back.

```
Input  →  [LLM]  →  Output
```

That's it. One question, one answer.

▶️ **Run the cell below**


In [ ]:
# Let's try it!
result = helpers.call_llm(
    "You're an AI assistant. Introduce yourself in 2-3 sentences."
)
print(result)

### 🎯 Exercise 1

Try changing the prompt below and running it. Ask the LLM something, anything!

✏️ **Edit and run the cell below**


In [ ]:
my_prompt = ""

result = helpers.call_llm(my_prompt)
print(result)

### Step 2: Structured Output

Getting free form text is nice, but what if we want **data** we can use in code?

We can ask the LLM to respond in a specific format (like JSON) so our program can parse and use the result using **Tools**.

▶️ **Run the cell below**


In [ ]:
# Define what fields we want back
fields = {
    "company_name": "string",
    "industry": "string",
    "founded_year": "number",
    "risk_level": "low | medium | high",
    "summary": "string",
}

# The LLM returns structured data matching our fields
result = helpers.call_llm_structured("Analyze Tesla as a company.", fields)
print(json.dumps(result, indent=2))

**Note:** Now the LLM's output isn't just text.. it's _data_ our code can work with.


### Step 3: Workflows

What if we need multiple steps? We can _chain_ LLM calls together, where the output of one becomes the input to the next.

```
Input → [LLM Call 1] → result₁ → [LLM Call 2] → result₂ → [LLM Call 3] → Final Output
```

**Insight:** Your _code_ decides the order. The LLM just executes each step.

▶️ **Run the cell below**


In [ ]:
# Process a customer support ticket through a fixed pipeline

ticket = """\
From: john.smith@email.com
Subject: Wrong item received

Hi, I'm writing about order ORD-99281 which I placed on Jan 15.

I ordered a blue laptop bag (SKU: LB-2847) but received a red one instead.
I need the correct item for a trip next week. This is the second time
this has happened with you guys. Very frustrated right now."""

print("🎫 SUPPORT TICKET:")
print(ticket)
print("\n" + "=" * 50 + "\n")

# STEP 1: Classify the ticket
classification = helpers.call_llm(
    f"Classify this ticket into exactly ONE word: shipping, billing, product_defect, wrong_item, or other.\n\n{ticket}\n\nCategory:"
)
category = classification.strip().lower()
print(f"📁 STEP 1 - CLASSIFICATION: {category}")

# STEP 2: Based on classification, extract the relevant fields
# (Different ticket types need different information extracted)
extraction = helpers.call_llm(
    f"""This is a {category} ticket. Extract ONLY the fields relevant to {category} issues:

For wrong_item: order_id, wrong_sku, expected_item
For shipping: order_id, tracking_number, delivery_date
For billing: order_id, amount, charge_date
For product_defect: order_id, sku, defect_description

Ticket:
{ticket}

Extract as a simple list:"""
)
print(f"\n📋 STEP 2 - EXTRACTION (for {category}):\n{extraction}")

# STEP 3: Generate response using the extracted data
response = helpers.call_llm(
    f"""Write a 3-sentence customer service reply for this {category} issue.

Extracted details:
{extraction}

Be apologetic and offer a concrete solution for {category} problems."""
)
print(f"\n📤 STEP 3 - RESPONSE:\n{response}")

### ⚠️ Compounding Errors

There's a catch with workflows. If each step is 90% accurate:

| Steps | Overall Accuracy |
| ----- | ---------------- |
| 1     | 90%              |
| 2     | 81%              |
| 3     | 73%              |
| 4     | 66%              |
| 5     | 59%              |

**The formula:** `P(correct) = (1 - error_rate)^steps`

The more steps you chain together, the more things can go wrong. This is the fundamental challenge of building with LLMs.

▶️ **Run the cell below**


In [ ]:
print("If each step is 90% accurate:\n")
for steps in range(1, 8):
    accuracy = 0.90**steps
    bar = "█" * int(accuracy * 30)
    print(f"  {steps} steps: {accuracy * 100:5.1f}% {bar}")

### 🎯 Exercise 2

Build your own multistep workflow. The topic is up to you.

Ideas:

- Step 1: Identify a location → Step 2: Analyze foot traffic → Step 3: Recommend products
- Step 1: Check inventory levels → Step 2: Analyze sales trends → Step 3: Create reorder list
- Or anything else!

✏️ **Edit and run the cell below**


In [ ]:
# Step 1
step1 = helpers.call_llm("")  # <-- Your prompt here
print(f"Step 1: {step1}\n")

# Step 2 (use step1's output)
step2 = helpers.call_llm("")  # <-- Your prompt here, reference {step1}
print(f"Step 2: {step2}\n")

# Step 3 (use previous outputs)
step3 = helpers.call_llm("")  # <-- Your prompt here, reference {step1} and/or {step2}
print(f"Step 3: {step3}")

### Step 4: Tool Calling

**Tool calling** lets us give the LLM a set of functions it can choose to invoke. The LLM decides _which_ tool to use and _what arguments_ to pass.

```
Input  →  [LLM]  →  "I want to use tool X with these arguments"  →  [Code runs tool]  →  Result
```

The LLM can't actually run the tools. It just tells us which one it wants. Our code executes it.

▶️ **Run the next two cells**


In [ ]:
DEMO_TOOLS = [
    helpers.tool(
        name="send_email",
        description="Send an email to someone",
        params=["to", "subject", "body"],
    ),
    helpers.tool(
        name="schedule_meeting",
        description="Schedule a meeting",
        params=["title", "time", "attendees"],
    ),
    helpers.tool(
        name="search_web",
        description="Search the web for information",
        params=["query"],
    ),
    helpers.tool(
        name="get_weather",
        description="Get the current weather for a location",
        params=["location"],
    ),
]

print("🔧 Available tools:")
for t in DEMO_TOOLS:
    params = list(t["input_schema"]["properties"].keys())
    print(
        f"   • {t['name']}({', '.join(params) if params else ''}) - {t['description']}"
    )

In [ ]:
scenario = """\
You are a personal assistant.
───────────────────────────────
Your boss says: "I need to meet with the London team next week.
Can you find out what the weather will be like and send them
an email to schedule something?"

What's your first action?"""

print("📋 SCENARIO:")
print(scenario)

response = helpers.call_llm_with_tools(scenario, DEMO_TOOLS)

print("\n🎬 THE LLM DECIDED TO:\n")
for block in response.content:
    if block.type == "tool_use":
        print(f"  📌 {block.name}({json.dumps(block.input)})")
    elif block.type == "text" and block.text.strip():
        print(f"  💭 {block.text[:150]}")

**Note:** We didn't tell the LLM _which_ tool to use. It analyzed the situation and chose the action on its own.

But we're still giving it one situation and getting one response. What if it could keep going?


## Step 5: The Agent Loop

Now we put it all together. An Agent is an LLM that:

1. **Observes** the current situation
2. **Thinks** about what to do
3. **Acts** using tools
4. **Repeats** until the goal is achieved

```
    ┌─────────┐
    │ OBSERVE │ ◄──────────┐
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │  THINK  │            │
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │   ACT   │────────────┘
    └─────────┘
```

The key difference from workflows:

- **Workflows:** Code decides the steps
- **Agents:** The LLM decides the steps

▶️ **Run the cell below**


In [ ]:
# The agent loop: observe → think → act → repeat
# This agent will take up to 3 actions to complete its task

task_state = """\
You are a research assistant preparing a brief for your manager.

TASK: Find information about SpaceX's latest launch and draft a summary email.

PROGRESS SO FAR:
- You haven't started yet

What do you do first?"""

actions = helpers.run_agent(
    company_name="Research Assistant",
    strategy="Be thorough but efficient",
    goal="Research SpaceX and draft a summary email to the manager",
    game_state=task_state,
    tools=DEMO_TOOLS,
    max_steps=3,
)

print("\n" + "=" * 40)
print("📋 ACTIONS TAKEN:")
for i, action in enumerate(actions, 1):
    print(f"  {i}. {action['tool']}({json.dumps(action['args'])})")

**Note:** We didn't tell the agent what to do but we told it _what to achieve_. It figured out the steps on its own.

|              | Workflows          | Agents            |
| ------------ | ------------------ | ----------------- |
| Who decides? | Code decides steps | LLM decides steps |
| Behavior     | Predictable        | Flexible          |
| Cost         | Cheaper & faster   | More expensive    |
| Best for     | Known paths        | Open-ended goals  |


# Part 2: The Vending Machine Game 🏪

Now let's put everything together and build something fun!

**You're going to build an AI agent that runs a vending machine business.**

Your agent will:

- 📧 **Email suppliers** to order inventory
- 💰 **Set prices** for products
- 📊 **Manage stock** to avoid running out
- 🎯 **Maximize profit** over 30 simulated days

## Game Rules

| Rule              | Details                              |
| ----------------- | ------------------------------------ |
| **Starting cash** | $500                                 |
| **Daily fee**     | $5/day operating cost                |
| **Products**      | Soda, Chips, Candy                   |
| **Goal**          | End with more money than you started |
| **Game over**     | Balance drops below $0 = bankrupt!   |

## Suppliers

| Supplier   | Prices (Soda/Chips/Candy) | Delivery | Trade-off            |
| ---------- | ------------------------- | -------- | -------------------- |
| QuickStock | $0.70 / $0.45 / $0.30     | 1 day    | Fast but expensive   |
| VendMart   | $0.60 / $0.40 / $0.25     | 1-2 days | Cheap but unreliable |
| BulkBarn   | $0.50 / $0.35 / $0.20     | 3 days   | Cheapest but slow    |

## How It Works

Each day:

1. Pay $5 daily fee
2. Deliveries arrive
3. Your agent takes actions (email suppliers, set prices)
4. Customers buy products (automatic, based on price)
5. Repeat for 30 days

**Lower prices = more customers, but thinner margins. Higher prices = fewer sales, but better margins.**

Now let's configure your agent!

✏️ **Edit and run the cell below**


In [ ]:
# YOUR AGENT'S CONFIGURATION
COMPANY_NAME = ""  # Give your vending machine business a name!

STRATEGY = ""  # e.g., "Focus on drinks. Keep prices competitive to drive volume."

# Personality sliders (1-10)
PRICING_STRATEGY = 5  # 1 = rock bottom prices, 10 = premium pricing
RISK_TOLERANCE = 5  # 1 = small safe orders, 10 = big bulk bets
NEGOTIATION_STYLE = 5  # 1 = accept first offer, 10 = haggle hard


▶️ **Run the next two cells**


In [ ]:
# Preview your agent configuration (validates your inputs)
helpers.show_agent(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)

In [ ]:
# This is the same concept from Part 1 - instructions that shape how the LLM behaves
system_prompt = helpers.build_system_prompt(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)
print(system_prompt)

### Test Your Agent

Let's give your agent a scenario and see what action it takes. This uses the same **tool calling** concept from Part 1.

▶️ **Run the cell below**


In [ ]:
test_scenario = """\
Day 10 · Balance: $387
───────────────────────────────────────
         Soda     Chips    Candy
Stock:   3        12       8
Price:   $1.99    $1.50    $1.00
Sales:   15 ⚠️    6        10

Suppliers:
• QuickStock: $0.70 / $0.45 / $0.30 (1-day)
• VendMart:   $0.60 / $0.40 / $0.25 (unreliable)
• BulkBarn:   $0.50 / $0.35 / $0.20 (3-day)
"""


# VENDING TOOLS - These are the actions your agent can take in the game (DO NOT CHANGE)
VENDING_TOOLS = [
    helpers.tool(
        name="send_email",
        description="Email a supplier to place an order. Include product names and quantities.",
        params=["to", "subject", "body"],
    ),
    helpers.tool(
        name="set_price",
        description="Set the retail price for a product (Soda, Chips, or Candy)",
        params=["product", "price"],
    ),
    helpers.tool(
        name="check_inventory",
        description="Check current stock levels",
    ),
    helpers.tool(
        name="check_balance",
        description="Check your bank balance",
    ),
]


helpers.test_agent(test_scenario, VENDING_TOOLS, system_prompt, COMPANY_NAME)

## 🚀 Launch the Simulation!

This runs your agent for 30 simulated days. Watch it make decisions, order inventory, and (hopefully) make money!

▶️ **Run the cell below**


In [ ]:
my_agent = {
    "company_name": COMPANY_NAME,
    "strategy": STRATEGY.strip(),
    "pricing_strategy": PRICING_STRATEGY,
    "risk_tolerance": RISK_TOLERANCE,
    "negotiation_style": NEGOTIATION_STYLE,
    "system_prompt": system_prompt,
}

launch_simulation(my_agent, API_KEY)

# 🎉 Wrap Up

### What We Built Today

| Step | Concept               | What We Did                                     |
| ---- | --------------------- | ----------------------------------------------- |
| 1    | **Single LLM Call**   | Sent a prompt, got a response                   |
| 2    | **Structured Output** | Got JSON data (product analysis), not just text |
| 3    | **Workflows**         | Chained calls: extract → calculate → respond    |
| 4    | **Tool Calling**      | Gave the LLM actions (email, set_price, etc.)   |
| 5    | **Agents**            | LLM decides its own steps to run a business     |

### Key Takeaway

**Add complexity only when you need it.**

Most real problems don't need agents. Many don't even need workflows. The right system is the simplest one that solves your problem.
